In [11]:
from Bio import Entrez, SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import pandas as pd
import re

import AS_motif_analysis as AS

In [4]:
# input significant alternative splicing (Skipped exons) evants with splicing sites coodinate
se_sig = pd.read_csv('/home/yu/R_studio_work_dir/AS_further_analysis/output/SE_sig.csv')
se_sig.iloc[:3,]

,ID,GeneID,geneSymbol,chr,strand,exonStart_0base,exonEnd,upstreamES,upstreamEE,downstreamES,...,FDR,IncLevelDifference,IncLevel1-1,IncLevel1-2,IncLevel1-3,IncLevel1-4,IncLevel2-1,IncLevel2-2,IncLevel2-3,IncLevel2-4
0,21433,ENSG00000183010,PYCR1,chr17,-,81934325,81934489,81932383,81933376,81934652,...,0.000000e+00,0.107,1.000,0.89,0.977,1.00,0.84,0.871,0.869,0.857
1,32017,ENSG00000186470,BTN3A2,chr6,+,26368177,26368267,26365227,26365352,26368564,...,0.000000e+00,-1.000,0.000,0.00,0.000,0.00,1.00,1.000,1.000,1.000
2,31416,ENSG00000151532,VTI1A,chr10,+,112448381,112448489,112446997,112447467,112448881,...,4.144463e-12,-0.807,0.189,0.00,0.304,0.28,1.00,1.000,1.000,1.000


In [21]:
## Calculate the length of upstream,dowstream exon and intron, and flanked exon, 
## and calcualte the genome coodinates in order to download from NCBI
se_sig_as_coord_len = pd.DataFrame(columns = ['ID','GeneID','geneSymbol','chr','strand',
                                              'seq_1_in_co1','seq_1_in_co2','seq_1_in_co3','seq_1_in_co4',
                                              'seq_2_in_co1','seq_2_in_co2','seq_2_in_co3','seq_2_in_co4',
                                              'seq_1_ex_co1','seq_1_ex_co2','seq_1_ex_co3','seq_1_ex_co4',
                                              'seq_2_ex_co1','seq_2_ex_co2','seq_2_ex_co3','seq_2_ex_co4'])

for i in range(0,len(se_sig)):
    
    if se_sig['strand'][i] == '+':
        se_sig_as_coord_len.loc[i,'seq_1_in_co2'] = se_sig.loc[i,'upstreamEE'].copy()
        se_sig_as_coord_len.loc[i,'seq_1_in_co3'] = se_sig.loc[i,'upstreamEE'].copy() + 1
        se_sig_as_coord_len.loc[i,'seq_2_in_co2'] = se_sig.loc[i,'exonStart_0base'].copy() - 1
        se_sig_as_coord_len.loc[i,'seq_2_in_co3'] = se_sig.loc[i,'exonStart_0base'].copy()

        se_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = se_sig.loc[i,'upstreamEE'].copy()
        se_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = se_sig.loc[i,'upstreamEE'].copy() + 1
        se_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = se_sig.loc[i,'downstreamES'].copy() - 1
        se_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = se_sig.loc[i,'downstreamES'].copy()
  
        if (se_sig.loc[i,'upstreamEE'] - se_sig.loc[i,'upstreamES'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_1_in_co1'] = se_sig.loc[i,'upstreamEE'].copy() - 150 + 1
            se_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = se_sig.loc[i,'upstreamEE'].copy() - 150 + 1                     
        else:
            se_sig_as_coord_len.loc[i,'seq_1_in_co1'] = se_sig.loc[i,'upstreamES']
            se_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = se_sig.loc[i,'upstreamES']
                                                      
        if (se_sig.loc[i,'exonStart_0base'] - se_sig.loc[i,'upstreamEE'] - 1) >= 300:
            se_sig_as_coord_len.loc[i,'seq_1_in_co4'] = se_sig.loc[i,'upstreamEE'] + 150
            se_sig_as_coord_len.loc[i,'seq_2_in_co1'] = se_sig.loc[i,'exonStart_0base'] - 150
        else:
            se_sig_as_coord_len.loc[i,'seq_1_in_co4'] = se_sig.loc[i,'upstreamEE'] + int((se_sig.loc[i,'exonStart_0base'] - se_sig.loc[i,'upstreamEE'] - 1)/2)
            se_sig_as_coord_len.loc[i,'seq_2_in_co1'] = se_sig.loc[i,'exonStart_0base'] - int((se_sig.loc[i,'exonStart_0base'] - se_sig.loc[i,'upstreamEE'] - 1)/2)                      
                                   
        if (se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1) >= 300:
            se_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = se_sig.loc[i,'upstreamEE'] + 150
            se_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = se_sig.loc[i,'downstreamES'] - 150                       
        else:
            se_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = se_sig.loc[i,'upstreamEE'] + int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1)/2)
            se_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = se_sig.loc[i,'downstreamES'] - int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1)/2)
        
        if (se_sig.loc[i,'exonEnd'] - se_sig.loc[i,'exonStart_0base'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_2_in_co4'] = se_sig.loc[i,'exonStart_0base'] + 150 -1
        else:
            se_sig_as_coord_len.loc[i,'seq_2_in_co4'] = se_sig.loc[i,'exonEnd']
        
        if (se_sig.loc[i,'downstreamEE'] - se_sig.loc[i,'downstreamES'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = se_sig.loc[i,'downstreamES'] + 150 -1
        else:
            se_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = se_sig.loc[i,'downstreamEE']
            
    else:
        se_sig_as_coord_len.loc[i,'seq_1_in_co2'] = se_sig.loc[i,'downstreamES'].copy()
        se_sig_as_coord_len.loc[i,'seq_1_in_co3'] = se_sig.loc[i,'downstreamES'].copy() - 1
        se_sig_as_coord_len.loc[i,'seq_2_in_co2'] = se_sig.loc[i,'exonEnd'].copy() + 1
        se_sig_as_coord_len.loc[i,'seq_2_in_co3'] = se_sig.loc[i,'exonEnd'].copy()

        se_sig_as_coord_len.loc[i,'seq_1_ex_co2'] = se_sig.loc[i,'downstreamES'].copy()
        se_sig_as_coord_len.loc[i,'seq_1_ex_co3'] = se_sig.loc[i,'downstreamES'].copy() - 1
        se_sig_as_coord_len.loc[i,'seq_2_ex_co2'] = se_sig.loc[i,'upstreamEE'].copy() + 1
        se_sig_as_coord_len.loc[i,'seq_2_ex_co3'] = se_sig.loc[i,'upstreamES'].copy()
        
        if (se_sig.loc[i,'downstreamEE'] - se_sig.loc[i,'downstreamES'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_1_in_co1'] = se_sig.loc[i,'downstreamES'].copy() + 150 - 1
            se_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = se_sig.loc[i,'downstreamES'].copy() + 150 - 1        
        else:
            se_sig_as_coord_len.loc[i,'seq_1_in_co1'] = se_sig.loc[i,'downstreamEE']
            se_sig_as_coord_len.loc[i,'seq_1_ex_co1'] = se_sig.loc[i,'downstreamEE']
            
            
        if (se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'exonEnd'] + 1) >= 300:
            se_sig_as_coord_len.loc[i,'seq_1_in_co4'] = se_sig.loc[i,'downstreamES'] - 150
            se_sig_as_coord_len.loc[i,'seq_2_in_co1'] = se_sig.loc[i,'exonEnd'] + 150
        else:
            se_sig_as_coord_len.loc[i,'seq_1_in_co4'] = se_sig.loc[i,'downstreamES'] - int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'exonEnd'] + 1)/2)
            se_sig_as_coord_len.loc[i,'seq_2_in_co1'] = se_sig.loc[i,'exonEnd'] + int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'exonEnd'] + 1)/2)                      
                                   
        if (se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1) >= 300:
            se_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = se_sig.loc[i,'downstreamES'] - 150
            se_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = se_sig.loc[i,'upstreamEE'] + 150                       
        else:
            se_sig_as_coord_len.loc[i,'seq_1_ex_co4'] = se_sig.loc[i,'downstreamES'] - int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1)/2)
            se_sig_as_coord_len.loc[i,'seq_2_ex_co1'] = se_sig.loc[i,'upstreamEE'] + int((se_sig.loc[i,'downstreamES'] - se_sig.loc[i,'upstreamEE'] - 1)/2)
        
        if (se_sig.loc[i,'exonEnd'] - se_sig.loc[i,'exonStart_0base'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_2_in_co4'] = se_sig.loc[i,'exonEnd'] - 150 + 1
        else:
            se_sig_as_coord_len.loc[i,'seq_2_in_co4'] = se_sig.loc[i,'exonStart_0base']
        
        if (se_sig.loc[i,'upstreamEE'] - se_sig.loc[i,'upstreamES'] + 1) >= 150:
            se_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = se_sig.loc[i,'upstreamEE'] -150 +1
        else:
            se_sig_as_coord_len.loc[i,'seq_2_ex_co4'] = se_sig.loc[i,'upstreamES']
                                   

se_sig_as_coord_len.iloc[:,:5] = se_sig.iloc[:,:5].copy()
se_sig_as_coord_len

,ID,GeneID,geneSymbol,chr,strand,seq_1_in_co1,seq_1_in_co2,seq_1_in_co3,seq_1_in_co4,seq_2_in_co1,...,seq_2_in_co3,seq_2_in_co4,seq_1_ex_co1,seq_1_ex_co2,seq_1_ex_co3,seq_1_ex_co4,seq_2_ex_co1,seq_2_ex_co2,seq_2_ex_co3,seq_2_ex_co4
0,21433,ENSG00000183010,PYCR1,chr17,-,81934745,81934652,81934651,81934570,81934571,...,81934489,81934340,81934745,81934652,81934651,81934502,81933526,81933377,81932383,81933227
1,32017,ENSG00000186470,BTN3A2,chr6,+,26365227,26365352,26365353,26365502,26368027,...,26368177,26368267,26365227,26365352,26365353,26365502,26368414,26368563,26368564,26369061
2,31416,ENSG00000151532,VTI1A,chr10,+,112447318,112447467,112447468,112447617,112448231,...,112448381,112448489,112447318,112447467,112447468,112447617,112448731,112448880,112448881,112450875
3,19134,ENSG00000248712,CCDC153,chr11,-,119194994,119194922,119194921,119194772,119194035,...,119193885,119193736,119194994,119194922,119194921,119194772,119190990,119190841,119190709,119190709
4,33281,ENSG00000169992,NLGN2,chr17,+,7408563,7408712,7408713,7408862,7412006,...,7412156,7412207,7408563,7408712,7408713,7408862,7414193,7414342,7414343,7414642
5,13887,ENSG00000177697,CD151,chr11,+,832873,833022,833023,833172,834379,...,834529,834591,832873,833022,833023,833172,835912,836061,836062,836062
6,39850,ENSG00000170919,TPT1-AS1,chr13,+,45379194,45379335,45379336,45379485,45379748,...,45379898,45380001,45379194,45379335,45379336,45379485,45382940,45383089,45383090,45383090
7,4782,ENSG00000079102,RUNX1T1,chr8,-,92102913,92102856,92102855,92102706,92076287,...,92076137,92075988,92102913,92102856,92102855,92102706,92017513,92017364,92017225,92017225
8,31920,ENSG00000141376,BCAS3,chr17,+,61380505,61380571,61380572,61380721,61388467,...,61388617,61388708,61380505,61380571,61380572,61380721,61391826,61391975,61391976,61392422
9,19138,ENSG00000147121,KRBOX4,chrX,+,46463197,46463308,46463309,46463458,46470991,...,46471141,46471185,46463197,46463308,46463309,46463458,46472599,46472748,46472749,46474788


In [16]:
## change AS ID to gene_as.count_as.type formate

list_example = list(se_sig_as_coord_len['geneSymbol']) # change the geneSymbol column to list
list_ID = [] # create a list to put new AS ID

list_ID.append(list_example[0] + '_' + str(1) + '_' + 'SE') # put the first element

## for loop, to count the AS numbers of the same gene in the lists before index of this AS events
for i in range(1, len(list_example)):
    count = list_example[0:i-1].count(list_example[i])
    if count == 0:
        list_ID.append(list_example[i] + '_' + str(1) + '_' + 'SE')
    else:
        list_ID.append(list_example[i] + '_' + str(count+1) + '_' + 'SE')
        
se_sig_as_coord_len.loc[:,'ID'] = list_ID

In [ ]:
se_sig_as_example = se_sig_as_coord_len.copy()

In [ ]:
# to get DNA sequences based on gene coorinates from NCB servers and save to fasta formate. 
# NOTE: this will send 508*6 queries to NCBI servers and do this in the weekend or unpeak time of USA
with open("AS_seqeunce.fasta", "w") as out_handle:
    for i in range(0,len(se_sig_as_example)):
        chr = se_sig_as_example.loc[i,'chr']
        chr_ID = human_chr_ID[chr]
    
        seq1_exon = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq1_exon_coord_1'],
                                     se_sig_as_example.loc[i,'seq1_exon_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq1_exon.id = se_sig_as_example.loc[i,'ID'] + '_seq1_exon'
        out_handle.write(seq1_exon.format("fasta"))

        
        seq1_intron = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq1_intron_coord_1'],
                                     se_sig_as_example.loc[i,'seq1_intron_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq1_intron.id = se_sig_as_example.loc[i,'ID'] + '_seq1_intron'
        out_handle.write(seq1_intron.format("fasta"))
        
        
        seq2_intron = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq2_intron_coord_1'],
                                     se_sig_as_example.loc[i,'seq2_intron_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq2_intron.id = se_sig_as_example.loc[i,'ID'] + '_seq2_intron'
        out_handle.write(seq2_intron.format("fasta"))        
    
    
        
        seq2_exon = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq2_exon_coord_1'],
                                     se_sig_as_example.loc[i,'seq2_exon_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq2_exon.id = se_sig_as_example.loc[i,'ID'] + '_seq2_exon'
        out_handle.write(seq2_exon.format("fasta"))  
        
        
        
        seq3_intron = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq3_intron_coord_1'],
                                     se_sig_as_example.loc[i,'seq3_intron_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq3_intron.id = se_sig_as_example.loc[i,'ID'] + '_seq3_intron'
        out_handle.write(seq3_intron.format("fasta")) 

        
        seq3_exon = get_dna_sequence(chr_ID,se_sig_as_example.loc[i,'seq3_exon_coord_1'],
                                     se_sig_as_example.loc[i,'seq3_exon_coord_2'],
                                     se_sig_as_example.loc[i,'strand'])
        seq3_exon.id = se_sig_as_example.loc[i,'ID'] + '_seq3_exon'
        out_handle.write(seq3_exon.format("fasta")) 

In [ ]:
#for genes in minus strand of chromosome, get the reverse complement sequence
Seq_records = SeqIO.parse('AS_seqeunce.fasta','fasta')
with open("AS_sequence_reverse_comp.fasta", "w") as out_handle2:
    for rec in Seq_records:
        match = re.match(r'([A-Za-z0-9\-\.]+)_',rec.id)
        gene_name = match.group(1)
        strand_df = se_sig.loc[se_sig['geneSymbol']== gene_name,['strand']]
        if strand_df.iloc[0,0] == '-':
            rec.description = rec.description + ' ' + 'Reverse_complement'
            rec.seq = rec.seq.reverse_complement()
        out_handle2.write(rec.format('fasta'))

In [15]:
#Input the fasta formate sequencing around splicing sites and do the alignment 
motif_count_array = np.array([])
motif_count = []
seq_names = []
motif_names = []

for i in AS.motif_dic.keys():
    count_motif_instances = 0
    match2 = re.match(r'([0-9\.]+)\-([A-Za-z\s]+)',i)
    for j in AS.motif_dic[i]:
        motif_names.append(match2.group() + str(count_motif_instances))
        count_motif_instances = count_motif_instances + 1

for rec in SeqIO.parse('AS_sequence_reverse_comp.fasta','fasta'):
    seq_names.append(rec.id)
    for i in AS.motif_dic.keys():
        for j in AS.motif_dic[i]:
            if len(rec.seq) >= len(j):
                count = rec.seq.count(j)
            else:
                count = -1           
            motif_count.append(count)
            
motif_count_array = np.array(motif_count).reshape(len(seq_names),len(motif_names))
motif_count_df = pd.DataFrame(motif_count_array, columns=motif_names, index=seq_names)

motif_count_df

,1.0-Exon enhancer0,2.0-Exon silencer0,3.0-Exon enhancer0,4.0-Exon silencer0,5.0-Exon silencer0,6.0-Exon enhancer0,7.0-Exon enhancer0,8.0-Exon enhancer0,9.0-Exon enhancer0,10.0-Intron enhancer0,...,443.0-Exon enhancer23,443.0-Exon enhancer24,443.0-Exon enhancer25,443.0-Exon enhancer26,443.0-Exon enhancer27,443.0-Exon enhancer28,443.0-Exon enhancer29,443.0-Exon enhancer30,443.0-Exon enhancer31,470.0-Exon enhancer0
PYCR1_1_SE_seq1_exon,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PYCR1_1_SE_seq1_intron,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PYCR1_1_SE_seq2_intron,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
PYCR1_1_SE_seq2_exon,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
PYCR1_1_SE_seq3_intron,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PYCR1_1_SE_seq3_exon,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTN3A2_1_SE_seq1_exon,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTN3A2_1_SE_seq1_intron,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTN3A2_1_SE_seq2_intron,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTN3A2_1_SE_seq2_exon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
